In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from typing import Optional, List, Any
from sklearn.datasets import make_classification

from mlrun.execution import MLClientCtx


def gen_class_data(
    context: MLClientCtx,
    n_samples: int,
    m_features: int,
    k_classes: int,
    header: Optional[List[str]],
    label_column: Optional[str] = "labels",
    weight: float = 0.5,
    random_state: int = 1,
    key: str = "classifier-data", 
    file_ext: str = "parquet",
    sk_params = {}
):
    """Create a binary classification sample dataset and save.
    If no filename is given it will default to:
    "simdata-{n_samples}X{m_features}.parquet".
    
    Additional scikit-learn parameters can be set using **sk_params, please see https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html for more details.
    
    :param context:       function context
    :param n_samples:     number of rows/samples
    :param m_features:    number of cols/features
    :param k_classes:     number of classes
    :param header:        header for features array
    :param label_column:  column name of ground-truth series
    :param weight:        fraction of sample negative value (ground-truth=0)
    :param random_state:  rng seed (see https://scikit-learn.org/stable/glossary.html#term-random-state)
    :param key:           key of data in artifact store
    :param file_ext:      (pqt) extension for parquet file
    :param sk_params:     additional parameters for `sklearn.datasets.make_classification`
    """
    features, labels = make_classification(
        n_samples=n_samples,
        n_features=m_features,
        weights=weight,
        n_classes=k_classes,
        random_state=random_state, 
        **sk_params)

    # make dataframes, add column names, concatenate (X, y)
    X = pd.DataFrame(features)
    if not header:
        X.columns = ["feat_" + str(x) for x in range(m_features)]
    else:
        X.columns = header

    y = pd.DataFrame(labels, columns=[label_column])
    data = pd.concat([X, y], axis=1)
    
    context.log_dataset(key, df=data, format=file_ext, index=False)

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or './'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("gen_class_data")

# add metadata (for templates and reuse)
fn.spec.default_handler = "gen_class_data"
fn.spec.description = "simulate classification data using scikit-learn"
fn.metadata.categories = ["simulators", "ml"]
fn.metadata.labels = {"author": "yjb", 'framework': 'sklearn'}

fn.export("function.yaml")

[mlrun] 2020-05-03 02:28:14,850 function spec saved to path: function.yaml


### test

In [7]:
if "V3IO_HOME" in list(os.environ):
    # if you are running this on the iguazio data fabric
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [13]:
from mlrun import NewTask    

task_params = {
    "name":        "tasks generate classification data", 
    "params" : {
        "n_samples"   : 250_000,
        "m_features"  : 64,
        "k_classes"   : 5,
        "weight"      : [0.1, 0.2, 0.3, 0.15],
        "sk_params"   : {"n_informative": 18},
        "file_ext"    : "pq"}}

In [14]:
from mlrun import run_local
run = run_local(
    NewTask(**task_params), 
    handler=gen_class_data,
    artifact_path=mlconf.artifact_path)

[mlrun] 2020-05-03 02:33:16,138 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 02:33:16,139 starting run tasks generate classification data uid=d8e44d59dd3542f699fb11bb04294592  -> http://mlrun-api:8080
[mlrun] 2020-05-03 02:33:16,201 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 02:33:21,830 log artifact classifier-data at /home/jovyan/data/classifier-data.parquet, size: 146031177, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...294592,0,May 03 02:33:16,completed,tasks generate classification data,kind=handlerowner=jovyanhost=jupyter-notebook-7c79d94f7d-c2lr9,,"n_samples=250000m_features=64k_classes=5weight=[0.1, 0.2, 0.3, 0.15]sk_params={'n_informative': 18}file_ext=pq",,classifier-data


to track results use .show() or .logs() or in CLI: 
!mlrun get run d8e44d59dd3542f699fb11bb04294592 --project default , !mlrun logs d8e44d59dd3542f699fb11bb04294592 --project default
[mlrun] 2020-05-03 02:33:22,264 run executed, status=completed


In [15]:
run = fn.run(
    NewTask(**task_params), artifact_path=mlconf.artifact_path)

[mlrun] 2020-05-03 02:33:28,756 starting run tasks generate classification data uid=1a80a8795ba742a3bbbc00e6e5feba9a  -> http://mlrun-api:8080
[mlrun] 2020-05-03 02:33:29,092 Job is running in the background, pod: tasks-generate-classification-data-t6b2q
[mlrun] 2020-05-03 02:33:51,222 log artifact classifier-data at /home/jovyan/data/classifier-data.parquet, size: 145991198, db: Y

[mlrun] 2020-05-03 02:33:51,857 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...feba9a,0,May 03 02:33:33,completed,tasks generate classification data,host=tasks-generate-classification-data-t6b2qkind=jobowner=jovyan,,"file_ext=pqk_classes=5m_features=64n_samples=250000sk_params={'n_informative': 18}weight=[0.1, 0.2, 0.3, 0.15]",,classifier-data


to track results use .show() or .logs() or in CLI: 
!mlrun get run 1a80a8795ba742a3bbbc00e6e5feba9a  , !mlrun logs 1a80a8795ba742a3bbbc00e6e5feba9a 
[mlrun] 2020-05-03 02:34:01,761 run executed, status=completed
